# Amazon Bedrock AgentCore Gatewayのセットアップ

## AgentCore Gateway用のCognito User Poolの作成

In [ ]:
# Import SDK and set up client
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
client = GatewayClient()

In [ ]:
# Retrieve the authorization configuration from the create response. When you create the gateway, specify it in the authorizer_config field
cognito_result = client.create_oauth_authorizer_with_cognito("my-gateway")
authorizer_configuration = cognito_result["authorizer_config"]

In [ ]:
authorizer_configuration

In [ ]:
import json
with open('agentcore_cognito_config.json', 'w') as f:
    json.dump(authorizer_configuration, f, indent=2, ensure_ascii=False)

## AgentCore Gatewayの作成

In [ ]:
# create the gateway.
gateway = client.create_mcp_gateway(
  name="github-mcp-gateway", # the name of the Gateway - if you don't set one, one will be generated.
  role_arn=None, # the role arn that the Gateway will use - if you don't set one, one will be created.
  authorizer_config=authorizer_configuration, # Variable from inbound authorization setup steps. Contains the OAuth authorizer details for authorizing callers to your Gateway (MCP only supports OAuth).
  enable_semantic_search=True, # enable semantic search.
)


## GatewayのTarget用Lambda関数の作成

In [ ]:
import boto3
import json 

iam = boto3.client('iam')

# Lambda実行ロールを作成
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

iam.create_role(
    RoleName='lambda-execution-role',
    AssumeRolePolicyDocument=json.dumps(trust_policy)
)

# 基本実行ポリシーをアタッチ
iam.attach_role_policy(
    RoleName='lambda-execution-role',
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)


In [ ]:
import zipfile
"""Lambda関数のデプロイメントパッケージを作成"""
zip_filename = 'lambda_function.zip'

with zipfile.ZipFile(zip_filename, 'w') as zip_file:
    # メインのLambda関数ファイルを追加
    zip_file.write('external_api_lambda.py', 'lambda_function.py')
    
    # 必要に応じて他のファイルも追加
    # zip_file.write('requirements.txt')

In [ ]:
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']

In [ ]:

"""Lambda関数をデプロイ"""
lambda_client = boto3.client('lambda')

# ZIPファイルを読み込み
with open(zip_filename, 'rb') as zip_file:
    zip_content = zip_file.read()

function_name = 'agentcore-external-api'

try:
    # 既存の関数を更新
    response = lambda_client.update_function_code(
        FunctionName=function_name,
        ZipFile=zip_content
    )
    print(f"Lambda関数を更新しました: {response['FunctionArn']}")
    
except lambda_client.exceptions.ResourceNotFoundException:
    # 新規作成
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.13',
        Role=f'arn:aws:iam::{account_id}:role/lambda-execution-role',
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': zip_content},
        Description='AgentCore External API Lambda',
        Timeout=300,
        MemorySize=128
    )
    print(f"arn: {response['FunctionArn']}")

# 一時ファイルを削除
os.remove(zip_filename)

## Targetの作成

In [ ]:

config = None

with open("lambda_tools_schema.json", 'rb') as file:
    schema = json.load(file)

    config = {
        "lambdaArn":response['FunctionArn'],
        "toolSchema": {
            "inlinePayload": schema
        }
    }

In [ ]:

# create a lambda target.
lambda_target = client.create_mcp_gateway_target(
    gateway={
        'gatewayId': 'github-mcp-gateway-htgcptu9xd',
    }, 
    name="github-mcp-function", # the name of the Target - if you don't set one, one will be generated.
    target_type="lambda", # the type of the Target - you will see other target types later in the tutorial.
    target_payload=config, # the target details - set this to define your own lambda if you pre-created one. Otherwise leave this None and one will be created for you.
    credentials=None, # you will see later in the tutorial how to use this to connect to APIs using API keys and OAuth credentials.
)
